In [2]:
import numpy as np
from random import random
from IPython.display import display, Math

In [3]:
def gradiente(f, xk, eps = 1e-6):
    size = len(xk)
    grad = np.zeros(size, dtype = float)
    for i in range(size):
        x_f = xk.copy(); 
        x_f[i] += eps
        x_b = xk.copy(); x_b[i] -= eps
        grad[i] = (f(x_f) - f(x_b)) / (2 * eps)
    return grad

In [4]:
def hessiana(f,xk, eps = 1e-3):
    size = len(xk)
    hess = np.zeros((size, size))
    for j in range(size):
        # Primera derivada
        dx_ff = xk.copy(); dx_ff[j] += eps;
        dx_bb = xk.copy(); dx_ff[j] += eps;
        grad_f = gradiente(f, dx_ff)
        grad_b = gradiente(f, dx_bb)
        for i in range(j+1):
            # Segunda derivada
            hess[i, j] = (grad_f[i] - grad_b[i]) / (2 * eps)
            hess[j, i] = (grad_f[i] - grad_b[i]) / (2 * eps)
    return hess

In [5]:
def Newton(f, xk, alpha, eps = 1e-6, max_iter = 1000): 
    for k in range(max_iter):
        grad = gradiente(f, xk)
        if(np.linalg.norm(grad) < eps): break
        hess = hessiana(f, xk)
        try:
            pk = np.linalg.solve(hess, -grad)
        except np.linalg.LinAlgError as err:
            print("La hessiana no es positiva definida")
            break
        xk = xk + alpha * pk
    return xk

In [6]:
def BLS(f, xk, alpha_0, pk, c):
    alpha = alpha_0
    while f(xk + alpha * pk) > f(xk) + c * alpha * gradiente(f, xk) @ pk:
        p = random()
        alpha = p * alpha
    return alpha

In [7]:
def LSN(f, xk):
    for k in range(100):
        Bk = hessiana(f, xk)
        try:
            pk = np.linalg.solve(Bk, - gradiente(f, xk))
        except np.linalg.LinAlgError as err:
            print("La hessiana no es positiva definida")
            break
        xk = xk + BLS(f, xk, 1, pk, 0.5) * pk
    return xk

In [8]:
def CAMI(A):
    beta = 0.001
    n = len(A)
    if min(np.diag(A)) > 0:
        t = 0
    else:
        t = -min(np.diag(A)) + beta
    for k in range(100):
        try:
            np.linalg.cholesky(A + t * np.identity(n))
        except np.linalg.LinAlgError as err:
            t = max(2 * t, beta)
        else:
            break
    return A + t * np.identity(n)

In [9]:
def LSNM(f, xk):
    for k in range(100):
        Bk = hessiana(f, xk)
        try:
            np.linalg.cholesky(Bk)
        except np.linalg.LinAlgError as err:
            Bk = CAMI(Bk)   
        pk = np.linalg.solve(Bk, - gradiente(f, xk))
        xk = xk + BLS(f, xk, 1, pk, 0.5) * pk
    return xk

In [10]:
def BFGS(f, xk, Hk, eps = 1e-6, max_iter = 10000):
    I = np.identity(len(xk))
    for k in range(max_iter):
        grad = gradiente(f, xk)
        if(np.linalg.norm(grad) < eps): break
        #Dirección de Descenso
        pk = -Hk @ grad
        #Calculamos el tamaño del paso
        alpha = BLS(f, xk, 1, pk, 0.5)
        #Nueva x
        sk = alpha*pk; sk_t = sk.transpose()
        xk = xk + sk
        #Calculamos yk y phi_k
        yk = gradiente(f,xk) - grad; yk_t = yk.transpose();
        if(yk_t@sk == 0): break
        phi_k = 1.0/(yk_t@sk)
        #Calculamos la nueva hessiana inversa
        Hk = (I - phi_k*sk@yk_t)@Hk@(I - phi_k*sk@yk_t) + phi_k * sk @ sk_t
    return xk

### Optimización de función de distancia

In [11]:
import math
import pandas as pd

In [12]:
data=pd.read_csv('crime_data.csv')
#Filtramos por fecha
fecha_inicio = '2019-12-25'
fecha_fin = '2019-12-25'
data = data.loc[((data['date']>=fecha_inicio) & (data['date']<=fecha_fin))]
#Nos quedamos con sólo longitud y latitud
data = data[{'lat','long'}]
#Lo convertimos en un numpy array
crimes = data.to_numpy()

In [13]:
n = 8 # Número de cameras
cameras = crimes[:8].copy()

In [14]:
# Regresa la distancia entre dos ountos que se encuentran  en lat y lon
def haversine(p1, p2):
    # Pasamos de decimales a radianes
    p1_lat = math.radians(p1[0]);p1_lon = math.radians(p1[1]);
    p2_lat = math.radians(p2[0]);p2_lon = math.radians(p2[1]);
    # formula de haversine
    h = math.sin((p2_lat-p1_lat)/2)**2 + math.cos(p1_lat) * math.cos(p2_lat) * math.sin((p2_lon-p1_lon)/2)**2
    c = 2 * math.asin(math.sqrt(h)) 
    r = 6371 # Radio de la tierra en km
    return c * r

In [15]:
def distancia_min(x, crimenes = crimes):
    #Suponemos x una función con latitud y longitud
    dist = 0
    for i in range(len(x)):
        for j in range(len(crimenes)):
            dist += haversine(x[i], crimenes[j])
        for j in range(len(x)):
            if(i != j):
                dist += 1/(haversine(x[i], x[j]))
    return dist 

In [16]:
#Distancia original
distancia_min(cameras)

2140.859064989902

In [59]:
def haversine(p1_a,p1_o, p2_a,p2_o):
    # Pasamos de decimales a radianes
    p1_lat = math.radians(p1_a);p1_lon = math.radians(p1_o);
    p2_lat = math.radians(p2_a);p2_lon = math.radians(p2_o);
    # formula de haversine
    h = math.sin((p2_lat-p1_lat)/2)**2 + math.cos(p1_lat) * math.cos(p2_lat) * math.sin((p2_lon-p1_lon)/2)**2
    c = 2 * math.asin(math.sqrt(h)) 
    r = 6371 # Radio de la tierra en km
    return c * r

cam_2 = np.zeros(2*len(cameras))
j = 0
for i in range(len(cameras)):
    cam_2[j] = (cameras[i])[0]
    cam_2[j+1] = (cameras[i])[1]
    j = j+2
    
cri_2 = np.zeros(2*len(crimes))
j = 0
for i in range(len(crimes)):
    cri_2[j] = (crimes[i])[0]
    cri_2[j+1] = (crimes[i])[1]
    j=j+2
    
def distancia_min(x, crimenes = cri_2):
    #Suponemos x una función con latitud y longitud
    dist = 0
    for i in range(int(len(x)/2)):
        for j in range(int(len(crimenes)/2)):
            dist += haversine(x[i],x[i+1], crimenes[j],crimenes[j+1])
        for j in range(int(len(x)/2)):
            if(i != j):
                dist += 1/(haversine(x[i],x[i+1], x[j],x[j+1]))
    return dist 

In [63]:
distancia_min(cam_2,cri_2)
LSNM(distancia_min, cam_2) 

array([-99.14583059,  19.26326885, -99.12509339,  19.2452067 ,
       -99.13185928,  19.27833462, -99.11662706,  19.26561897,
       -99.10592306,  19.3758    , -99.09761   ,  19.35711   ,
       -99.06398   ,  19.31785   , -99.13508   ,  19.51575   ])

In [64]:
x = np.array([[-99.14583059,  19.26326885], [-99.12509339,  19.2452067] ,
       [-99.13185928,  19.27833462], [-99.11662706,  19.26561897],
       [-99.10592306,  19.3758]    , [-99.09761   ,  19.35711]   ,
       [-99.06398   ,  19.31785]   , [-99.13508   ,  19.51575]])

In [65]:
def haversine(p1, p2):
    # Pasamos de decimales a radianes
    p1_lat = math.radians(p1[0]);p1_lon = math.radians(p1[1]);
    p2_lat = math.radians(p2[0]);p2_lon = math.radians(p2[1]);
    # formula de haversine
    h = math.sin((p2_lat-p1_lat)/2)**2 + math.cos(p1_lat) * math.cos(p2_lat) * math.sin((p2_lon-p1_lon)/2)**2
    c = 2 * math.asin(math.sqrt(h)) 
    r = 6371 # Radio de la tierra en km
    return c * r

def distancia_min(x, crimenes = crimes):
    #Suponemos x una función con latitud y longitud
    dist = 0
    for i in range(len(x)):
        for j in range(len(crimenes)):
            dist += haversine(x[i], crimenes[j])
        for j in range(len(x)):
            if(i != j):
                dist += 1/(haversine(x[i], x[j]))
    return dist 

In [66]:
# Distancia modificada
distancia_min(x)

2076.45792203015